In [1]:
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import torch
import json
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
import torchvision.models as models

import urllib.request
from PIL import Image
from google.colab import files

import torch
import torch.nn as nn
from torchvision import transforms


from captum.attr import IntegratedGradients,DeepLift
from captum.attr import visualization as viz

In [4]:
#Downloads the list of classes/labels for ImageNet datase
import urllib.request

url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
save_path = "drive/MyDrive/imagenet_class_index.json"  # Replace with the desired save path

urllib.request.urlretrieve(url, save_path)

('drive/MyDrive/imagenet_class_index.json',
 <http.client.HTTPMessage at 0x7f6406f7d390>)

In [5]:
import drive.MyDrive.pytorch_viz as viz

with open("drive/MyDrive/imagenet_class_index.json") as json_data:
    idx_to_labels = json.load(json_data)


In [6]:
preprocess = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize the image to match the model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Download the image from the internet
image_url = "https://images.unsplash.com/photo-1495360010541-f48722b34f7d?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8NHx8Y2F0fGVufDB8fDB8fHww&auto=format&fit=crop&w=500&q=60"
image_path = "cat.jpg"
urllib.request.urlretrieve(image_url, image_path)

# Load and preprocess the input image
image = Image.open(image_path)
image_tensor = preprocess(image).unsqueeze(0)

In [7]:
# Download models
model_resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model_inception = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model_inception.eval()
model_vgg = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [36]:
model_resnet.eval()
with torch.no_grad():
    logits_resnet = model_resnet(image_tensor)

In [37]:
prediction_score, target_resnet = torch.topk(torch.softmax(logits_resnet, dim=1), k=5)


In [40]:
target_resnet[0][0]

tensor(281)

In [10]:
# Print the top 5 labels and their probabilities
for prob, idx in zip(prediction_score[0], target_resnet[0]):
  print(idx_to_labels[str(idx.item())][1]+" "+ str(prob.squeeze().item()))

tabby 0.3873326778411865
tub 0.1333470344543457
bathtub 0.11723610758781433
Egyptian_cat 0.10701601952314377
tiger_cat 0.10457748174667358


In [11]:
model_vgg.eval()
with torch.no_grad():
    logits_vgg = model_vgg(image_tensor)

In [12]:
prediction_score, target_vgg = torch.topk(torch.softmax(logits_vgg, dim=1), k=5)


In [13]:
# Print the top 5 labels and their probabilities
for prob, idx in zip(prediction_score[0], target_vgg[0]):
  print(idx_to_labels[str(idx.item())][1]+" "+ str(prob.squeeze().item()))

tabby 0.2592127323150635
tub 0.21564650535583496
bathtub 0.16611334681510925
Egyptian_cat 0.10538455843925476
tiger_cat 0.09683449566364288


In [14]:
model_inception.eval()
with torch.no_grad():
    logits_inception = model_inception(image_tensor)

In [15]:
prediction_score, target_inception = torch.topk(torch.softmax(logits_vgg, dim=1), k=5)


In [26]:
torch.topk(prediction_score, 1).indices.item()

0

In [16]:
# Print the top 5 labels and their probabilities
for prob, idx in zip(prediction_score[0], target_inception[0]):
  print(idx_to_labels[str(idx.item())][1]+" "+ str(prob.squeeze().item()))

tabby 0.2592127323150635
tub 0.21564650535583496
bathtub 0.16611334681510925
Egyptian_cat 0.10538455843925476
tiger_cat 0.09683449566364288


## Lets start interpreting the models


1.   Using Integrated Gradient
2.   Using DEEPLIFT



In [17]:
'''
create a baseline against which the model is going to get tallied.
here, we will create a baseline of all zeros
'''
baseline = torch.zeros(image_tensor.shape)

In [18]:
baseline.shape ## [1, 3, 224, 224] means one image with 3 channels and 224x224 resolution

torch.Size([1, 3, 299, 299])

Integrated Gradient


In [74]:
IG_resnet = IntegratedGradients(model_resnet)
attrs_ig_resnet = IG_resnet.attribute(image_tensor, target=target_resnet[0][0],n_steps=10)

IG_vgg = IntegratedGradients(model_vgg)
attrs_ig_vgg = IG_vgg.attribute(image_tensor, target=target_vgg[0][0],n_steps=10)

IG_inception = IntegratedGradients(model_inception)
attrs_ig_inception = IG_inception.attribute(image_tensor, target=target_inception[0][0],n_steps=10)

In [46]:
## define a custom heatmap (not necessary, can use predefined heatmap too)
## have a look at the original image and model predicitions
from matplotlib.colors import LinearSegmentedColormap
default_cmap = LinearSegmentedColormap.from_list('custom blue',
                                                 [(0, '#ffffff'),
                                                  (0.25, '#000000'),
                                                  (1, '#000000')], N=256)

In [80]:
viz.visualize_image_attr_multiple(np.transpose(attrs_lrp_vgg.squeeze().cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image_tensor.squeeze().cpu().detach().numpy(), (1,2,0)),
                             ["original_image", "heat_map"],
                             ["all", "positive"],
                             cmap=default_cmap,
                             show_colorbar=True,
                             outlier_perc=1)

(<Figure size 500x500 with 4 Axes>, array([<Axes: >, <Axes: >], dtype=object))

In [92]:
viz.visualize_img(attrs_ig_resnet, image_tensor, default_cmap,  titles=['Integrated Gradient on ResNet', ''])
viz.visualize_img(attrs_ig_vgg, image_tensor, default_cmap,  titles=['Integrated Gradient on VGG', ''])
viz.visualize_img(attrs_ig_inception, image_tensor, default_cmap,  titles=['Integrated Gradient on Inception', ''])

(<Figure size 500x500 with 2 Axes>,
 array([<Axes: title={'center': 'Integrated Gradient on Inception'}>,
        <Axes: >], dtype=object))

In [95]:
# DL_resnet = DeepLift(model_resnet)
# attrs_dl_resnet = DL_resnet.attribute(image_tensor, target=target_resnet[0][0], )

DL_vgg = DeepLift(model_vgg)
attrs_dl_vgg = DL_vgg.attribute(image_tensor, target=target_vgg[0][0], )

DL_inception = DeepLift(model_inception)
attrs_dl_inception = DL_inception.attribute(image_tensor, target=target_inception[0][0], )

In [96]:
# viz.visualize_img(attrs_dl_resnet, image_tensor, default_cmap,  titles=['Integrated Gradient on ResNet', ''])
viz.visualize_img(attrs_dl_vgg, image_tensor, default_cmap,  titles=['Integrated Gradient on VGG', ''])
viz.visualize_img(attrs_dl_inception, image_tensor, default_cmap,  titles=['Integrated Gradient on Inception', ''])

(<Figure size 500x500 with 2 Axes>,
 array([<Axes: title={'center': 'Integrated Gradient on Inception'}>,
        <Axes: >], dtype=object))